<a href="https://colab.research.google.com/github/sudeshna0501/twitter-sentiment-analysis/blob/main/LSTM_%2B_CNN_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/My Drive/Tweets.csv')

In [ ]:
X= df['text']
Y= df['airline_sentiment']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import np_utils

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(df['airline_sentiment'])

In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [ ]:
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

In [ ]:
X = X.apply(clean_doc)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = \
train_test_split(X, dummy_y,test_size=0.2)

In [ ]:
X_train_lstm = X_train

In [ ]:
def joining(ele):
 return " ".join(str(v) for v in ele)

In [ ]:
char_X=X_train_lstm.apply(joining)


In [ ]:
snt_emb = X_train_lstm.apply(joining)

In [ ]:
def split(word):
    return list(word)

In [ ]:
char_X=char_X.apply(split)

In [ ]:
char_X

11737    [u, s, a, i, r, w, a, y, s,  , c, o, o, l,  , ...
3311     [u, n, i, t, e, d,  , D, o, n, n, a,  , J, u, ...
7270     [J, e, t, B, l, u, e,  , t, h, a, t, s,  , o, ...
6791     [M, o, r, e,  , l, i, k, e,  , B, a, e, J, e, ...
6106     [S, o, u, t, h, w, e, s, t, A, i, r,  , w, i, ...
                               ...                        
14272    [n, o, t, h, e, l, p, f, u, l,  , M, T,  , A, ...
1716     [u, n, i, t, e, d,  , f, l, i, g, h, t,  , d, ...
9271     [U, S, A, i, r, w, a, y, s,  , y, e, t,  , F, ...
5045     [S, o, u, t, h, w, e, s, t, A, i, r,  , H, U, ...
8841     [J, e, t, B, l, u, e,  , p, l, s,  , f, i, n, ...
Name: text, Length: 11712, dtype: object

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate


In [ ]:
# fit a tokenizer
def create_tokenizer(lines):
 tokenizer = Tokenizer()
 tokenizer.fit_on_texts(lines)
 return tokenizer


# calculate the maximum document length
def max_length(lines):
 return max([len(s.split()) for s in lines])


# encode a list of lines
def encode_text(tokenizer, lines, length):
 # integer encode
 encoded = tokenizer.texts_to_sequences(lines)
 # pad encoded sequences
 padded = pad_sequences(encoded, maxlen=length, padding='post')
 return padded




#for calculating length
def joining(ele):
 return " ".join(str(v) for v in ele)


In [ ]:
#for cnn

# create tokenizer
tokenizer_cnn= create_tokenizer(X_train)
# calculate max document length
joinned_train_cnn=X_train.apply(joining)
length_cnn= max_length(joinned_train_cnn)
# calculate vocabulary size
vocab_size_cnn = len(tokenizer_cnn.word_index) + 1
print('Max document length: %d' % length_cnn)
print('Vocabulary size: %d' % vocab_size_cnn)
# encode data
trainX_cnn= encode_text(tokenizer_cnn, X_train, length_cnn)
print(trainX_cnn.shape)


Max document length: 23
Vocabulary size: 11578
(11712, 23)


In [ ]:
#for lstm

# create tokenizer
tokenizer_lstm = create_tokenizer(char_X)
# calculate max document length
joinned_train_lstm =char_X.apply(joining)
length_lstm = max_length(joinned_train_lstm)
# calculate vocabulary size
vocab_size_lstm = len(tokenizer_lstm.word_index) + 1
print('Max document length: %d' % length_lstm)
print('Vocabulary size: %d' % vocab_size_lstm)
# encode data
trainX_lstm = encode_text(tokenizer_lstm, char_X, length_lstm)
print(trainX_lstm.shape)

Max document length: 127
Vocabulary size: 34
(11712, 127)


In [ ]:
# define the model
def define_model(length_lstm, length_cnn, vocab_size_lstm, vocab_size_cnn):
 # channel 1
 inputs1 = Input(shape=(length_lstm,))
 embedding1 = Embedding(vocab_size_lstm, 128, input_length=length_lstm)(inputs1)
 lstm1= (LSTM(100))(embedding1)
 flat1 =(Dropout(0.5))(lstm1)
 # channel 2
 inputs2 = Input(shape=(length_cnn,))
 embedding2 = Embedding(vocab_size_cnn, 100)(inputs2)
 conv2 = Conv1D(filters=16, kernel_size=2, activation='relu')(embedding2)
 drop2 = Dropout(0.5)(conv2)
 pool2 = MaxPooling1D(pool_size=2)(drop2)
 flat2 = Flatten()(pool2)

 #channel 3
 inputs3 = Input(shape=(length_cnn,))
 embedding3 = Embedding(vocab_size_cnn, 100)(inputs3)
 conv3 = Conv1D(filters=32, kernel_size=3, activation='relu')(embedding3)
 drop3 = Dropout(0.5)(conv3)
 pool3 = MaxPooling1D(pool_size=2)(drop3)
 flat3 = Flatten()(pool3)



#flat1, flat2, flat3
 # merge
 merged = concatenate([flat1, flat2, flat3])
 # interpretation
 dense1 = Dense(300, activation='relu')(merged)
 dense2 = Dense(100, activation='relu')(dense1)
 dense3 = Dense(20, activation='relu')(dense2)
 dense4 = Dense(10, activation='relu')(dense3)
 outputs = Dense(3, activation='softmax')(dense4)

#inputs1, inputs2,inputs3
 model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
 # compile
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # summarize
 print(model.summary())
 plot_model(model, show_shapes=True, to_file='multichannel.png')
 return model

In [ ]:

# define model
model = define_model(length_lstm, length_cnn, vocab_size_lstm,vocab_size_cnn)
print(model.summary())



# fit model
model.fit([trainX_lstm, trainX_cnn, trainX_cnn], y_train, epochs=5, batch_size=20)
# save the model
model.save('model.h5')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 23, 100)      1157800     ['input_5[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 23, 100)      1157800     ['input_6[0][0]']                
                                                                                            

In [ ]:
X_test_lstm=X_test

In [ ]:
def joining(ele):
 return " ".join(str(v) for v in ele)

In [ ]:
char_X_test=X_test_lstm.apply(joining)

In [ ]:
def split(word):
    return list(word)

In [ ]:
char_X_test=char_X_test.apply(split)

In [ ]:
sentence_vectors_test = X_test_lstm.apply(joining)

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model



# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])

# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

  #for calculating length
def joining(ele):
  return " ".join(str(v) for v in ele)


#for cnn

# create tokenizer
tokenizer_cnn= create_tokenizer(X_train)
# calculate max document length
joinned_train_cnn=X_train.apply(joining)
length_cnn= max_length(joinned_train_cnn)
# calculate vocabulary size
vocab_size_cnn = len(tokenizer_cnn.word_index) + 1
print('Max document length: %d' % length_cnn)
print('Vocabulary size: %d' % vocab_size_cnn)
# encode data
trainX = encode_text(tokenizer_cnn, X_train, length_cnn)
testX = encode_text(tokenizer_cnn, X_test, length_cnn)
print(trainX.shape)

#for lstm

# create tokenizer
tokenizer_lstm = create_tokenizer(char_X)
# calculate max document length
joinned_train_lstm =char_X.apply(joining)
length_lstm = max_length(joinned_train_lstm)
# calculate vocabulary size
vocab_size_lstm = len(tokenizer_lstm.word_index) + 1
print('Max document length: %d' % length_lstm)
print('Vocabulary size: %d' % vocab_size_lstm)
# encode data
trainX_lstm = encode_text(tokenizer_lstm, char_X, length_lstm)
testX_lstm= encode_text(tokenizer_lstm, char_X_test, length_lstm)
print(trainX_lstm.shape)



#testLabels = list(test_data['sentiment'])
# load the model
model = load_model('model.h5')

# evaluate model on training dataset
loss, acc = model.evaluate([trainX_lstm, trainX, trainX], y_train, verbose=0)
print('Train Accuracy: %f' % (acc*100))

# evaluate model on test dataset dataset
loss, acc = model.evaluate([testX_lstm, testX, testX], y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Max document length: 23
Vocabulary size: 11578
(11712, 23)
Max document length: 127
Vocabulary size: 34
(11712, 127)
Train Accuracy: 98.326504
Test Accuracy: 76.707649
